In [1]:
import pandas as pd
import numpy as np
import datetime as dt


In [2]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 95%; }
</style>
"""))

In [3]:
#### INPUTS #####
# location of data
file_loc = 'New Folder/tech_challenge_indication_data.xlsx'

# enter year, month, day
effective_date = dt.datetime(2022, 7, 1) 

In [4]:
# import Excel sheets
prem_raw = pd.read_excel(file_loc, sheet_name=0)
loss_raw = pd.read_excel(file_loc, sheet_name=1)
ldf = pd.read_excel(file_loc, sheet_name=2)
ptrend = pd.read_excel(file_loc, sheet_name=3)
ltrend = pd.read_excel(file_loc, sheet_name=4)
lae = pd.read_excel(file_loc, sheet_name=5).set_index('Unnamed: 0').T
exp = pd.read_excel(file_loc, sheet_name=6).set_index('Expense').T
cred = pd.read_excel(file_loc, sheet_name=7).set_index('Unnamed: 0').T


In [5]:
# calculate trending time periods
coverages = list(prem_raw.columns[1:11])
prem_raw['trend_yrs'] = (prem_raw.Year_Ending.max()-prem_raw.Year_Ending).astype('timedelta64[D]')/365
loss_raw['trend_yrs'] = (loss_raw.Year_Ending.max()-prem_raw.Year_Ending).astype('timedelta64[D]')/365

prem_ftrend = (effective_date-prem_raw.Year_Ending.max()).days/365 + 0.5
loss_ftrend = (effective_date-loss_raw.Year_Ending.max()).days/365 + 1

In [6]:
# apply trends and LDFs
# premium trend
prem = prem_raw
for i in coverages:
    prem[i] = prem_raw[i]*(1+ptrend.iloc[0,coverages.index(i)+1])**(prem_raw['trend_yrs'])*(1+ptrend.iloc[1,coverages.index(i)+1])**prem_ftrend

# LDFs
loss_ult = loss_raw
for i in coverages:
    loss_ult[i] = loss_raw[i]*(1+ldf.iloc[0,coverages.index(i)+1])

# loss trend
loss = loss_ult
for i in coverages:
    loss[i] = loss_raw[i]*(1+ltrend.iloc[0,coverages.index(i)+1])**(loss_raw['trend_yrs'])*(1+ltrend.iloc[1,coverages.index(i)+1])**loss_ftrend


In [10]:
# get total premiums and losses, attach all other factors, preform calculations

totals = pd.concat([prem.sum(), loss.sum(), lae, exp, cred], axis=1).rename(columns={0:'Premium', 1:'Losses'})
totals.Premium = totals.Premium.astype('int')
totals.Losses = totals.Losses.astype('int')
totals = totals[0:10]
totals['Loss_ratio'] = totals['Losses']*totals['LAE_loads']/totals['Premium']
totals['Variable_Expenses'] = totals['Commission']+totals['General']+totals['Other_Acquisition']+totals['Taxes_License_and_Fees']
totals['PLR'] = (1-totals['Variable_Expenses'])
totals['Indication'] = totals['Loss_ratio']/totals['PLR']
totals['Cred_Indication'] = totals['Indication']*totals['Credibility'] + (1-totals['Credibility'])
totals

,Premium,Losses,LAE_loads,Commission,General,Other_Acquisition,Taxes_License_and_Fees,Credibility,Loss_ratio,Variable_Expenses,PLR,Indication,Cred_Indication
Coverage_01,1315470,2034690,1.20,0.0,0.03,0.04,0.02,0.33,1.856088,0.09,0.91,2.039657,1.343087
Coverage_02,1092800,2962144,1.15,0.0,0.03,0.04,0.02,0.33,3.117190,0.09,0.91,3.425484,1.800410
Coverage_03,977543,2226216,1.40,0.0,0.03,0.04,0.02,0.33,3.188302,0.09,0.91,3.503629,1.826197
Coverage_04,861583,1652937,1.50,0.0,0.03,0.04,0.02,0.33,2.877733,0.09,0.91,3.162344,1.713573
Coverage_05,771539,1662489,1.20,0.0,0.03,0.04,0.02,0.33,2.585724,0.09,0.91,2.841455,1.607680
Coverage_06,585977,1577806,1.05,0.0,0.03,0.04,0.02,0.25,2.827238,0.09,0.91,3.106855,1.526714
Coverage_07,389191,1211575,1.05,0.0,0.03,0.04,0.02,0.25,3.268713,0.09,0.91,3.591993,1.647998
Coverage_08,288680,743832,1.00,0.0,0.03,0.04,0.02,0.15,2.576666,0.09,0.91,2.831501,1.274725
Coverage_09,304611,835225,1.00,0.0,0.03,0.04,0.02,0.15,2.741940,0.09,0.91,3.013121,1.301968
Coverage_10,196198,468267,1.00,0.0,0.03,0.04,0.02,0.15,2.386706,0.09,0.91,2.622754,1.243413


In [ ]:
totals = pd.concat([prem_raw.sum(), prem.sum(), loss_raw.sum(), loss_ult.sum(), loss.sum(), lae, exp, cred], axis=1).rename(
    columns={0:'Earned_Premium', 1:'Trended_Premium', 2:'Raw_Losses', 3:'Ultimate_Losses', 4:'Trended_and_Ultimate_Losses'})
totals.Trended_Premium = totals.Trended_Premium.astype('int')
totals.Trended_and_Ultimate_Losses = totals.Trended_and_Ultimate_Losses.astype('int')
totals = totals[0:10]
totals['Loss_ratio'] = totals['Trended_and_Ultimate_Losses']*totals['LAE_loads']/totals['Trended_Premium']
